# Mapping Agricultural-Driven Deforestation in Central Kalimantan with Remote Sensing

Lu Yii Wong 

May 2025

## Part 1: Setup and Data Aquisition 

In [1]:
# Setup Libraries
import ee
import geemap
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
import geopandas as gpd
from datetime import datetime

In [2]:
# Authenticate GEE
ee.Authenticate()
ee.Initialize(project='ee-wongluyii')

In [7]:
# Pull Sentinel-2 Imagary using GEE

# Load Boundary for Area of Interest (Pulang Pisau)
gdf = gpd.read_file("data/Pulang_Pisau_boundary.geojson").to_crs(epsg=4326)
roi = geemap.geopandas_to_ee(gdf)

# Load Sentinel-2 SF Image (2A)
sentinel2 = (ee.ImageCollection("COPERNICUS/S2_SR")
             .filterDate("2018-12-01", "2019-12-31")
             .filterBounds(roi)
             .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 30)))  # Optional: <30% cloud cover

# Function to mask clouds and cirrus using QA60 bitmask
def maskS2clouds(image):
    qa = image.select("QA60")
    cloudBitMask = 1 << 10  # Bit 10: clouds
    cirrusBitMask = 1 << 11 # Bit 11: cirrus
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(
           qa.bitwiseAnd(cirrusBitMask).eq(0))
    return image.updateMask(mask).divide(10000).copyProperties(image, ["system:time_start"])

In [8]:
# Apply cloud masking 
sentinel2_clean = sentinel2.map(maskS2clouds)

# Select bands 
sentinel2_selected = sentinel2_clean.select(["B8", "B12", "B4", "B11", "B3", "B2", "B5", "B6", "B7", "B8A"])

In [9]:
# Create Visual Parameters and Display map
viz_params = {
    "bands": ["B4", "B3", "B2"],
    "min": 0.0,
    "max": 0.3
}

Map = geemap.Map()
Map.centerObject(roi, 9)
Map.addLayer(sentinel2_selected.median().clip(roi), viz_params, "Sentinel-2 Median (2018-2019)")
Map.addLayer(roi, {}, "Pulang Pisau Boundary")
Map

Map(center=[-2.7158640396563944, 113.95461205914583], controls=(WidgetControl(options=['position', 'transparen…

## Calculate Normalized Burn Ratio (NBR) 

In [10]:
# Create a function to calculate NBR for each Sentinel-2 image
def calculate_nbr(image):
    # NBR = (NIR - SWIR2) / (NIR + SWIR2)
    nbr = image.normalizedDifference(["B8", "B12"]).rename("NBR")
    return image.addBands(nbr)

# Apply the NBR function across the cleaned Sentinel-2 collection
sentinel2_with_nbr = sentinel2_selected.map(calculate_nbr)

# Only keep the NBR band for easier processing later
nbr_collection = sentinel2_with_nbr.select("NBR")

# Visualize the median NBR across the whole period
Map = geemap.Map()
Map.centerObject(roi, 9)
Map.addLayer(nbr_collection.median().clip(roi), {"min": -0.5, "max": 1, "palette": ["white", "green", "black"]}, "Median NBR (2018-2019)")
Map.addLayer(roi, {}, "Pulang Pisau Boundary")
Map

Map(center=[-2.7158640396563944, 113.95461205914583], controls=(WidgetControl(options=['position', 'transparen…

In [11]:
# Check number of images in the filtered Sentinel-2 ImageCollection
n_images = nbr_collection.size().getInfo()
print(f"Number of Sentinel-2 images with NBR in the collection: {n_images}")

# Get list of image dates
timestamps = nbr_collection.aggregate_array("system:time_start").getInfo()
dates = [datetime.utcfromtimestamp(t / 1000).strftime('%Y-%m-%d') for t in timestamps]

# Create a DataFrame
dates_df = pd.DataFrame(dates, columns=["Acquisition Date"])
dates_df.index += 1
print(dates_df)

Number of Sentinel-2 images with NBR in the collection: 234


In [32]:
# Count number of unique acquisition dates
unique_dates = dates_df["Acquisition Date"].nunique()
print(f"Number of unique dates: {unique_dates}")

# Print sorted unique dates
unique_dates_list = sorted(dates_df["Acquisition Date"].unique())

for date in unique_dates_list:
    print(date)

Number of unique dates: 64
2018-12-30
2019-02-08
2019-02-15
2019-02-20
2019-02-23
2019-02-25
2019-03-15
2019-03-17
2019-03-20
2019-03-22
2019-04-01
2019-04-06
2019-04-21
2019-04-26
2019-05-01
2019-05-04
2019-05-06
2019-05-09
2019-05-16
2019-05-24
2019-06-05
2019-06-23
2019-06-28
2019-06-30
2019-07-05
2019-07-08
2019-07-13
2019-07-15
2019-07-23
2019-07-25
2019-07-28
2019-07-30
2019-08-02
2019-08-04
2019-08-09
2019-08-12
2019-08-14
2019-08-17
2019-08-19
2019-08-22
2019-08-29
2019-09-01
2019-09-06
2019-09-08
2019-09-11
2019-09-13
2019-09-18
2019-09-23
2019-09-28
2019-10-08
2019-10-13
2019-10-23
2019-10-28
2019-11-05
2019-11-07
2019-11-10
2019-11-15
2019-11-17
2019-11-22
2019-11-25
2019-11-27
2019-12-05
2019-12-20
2019-12-27


## Pre- and Post-Seaonal Fire Composites
The initial method using the moving window does not fit with the data that I have for Pulang Pisau. 

In [ ]:
# Pre-Fire Composite (before dry season fires)
pre_fire = (sentinel2_selected
            .filterDate("2019-01-01", "2019-06-30")
            .median()
            .clip(roi))

# Post-Fire Composite (after fire season)
post_fire = (sentinel2_selected
             .filterDate("2019-09-01", "2019-12-31")
             .median()
             .clip(roi))

# Calculate NBR for Pre- and Post-Fire Composites
pre_fire_nbr = pre_fire.normalizedDifference(["B8", "B12"]).rename("NBR")
post_fire_nbr = post_fire.normalizedDifference(["B8", "B12"]).rename("NBR")

# Calculate dNBR
dnbr_simple = pre_fire_nbr.subtract(post_fire_nbr).rename("dNBR")

# Burn mask (Only show areas with dNBR > 0.2)
burn_mask = dnbr_simple.gt(0.2)

# Visualization
Map = geemap.Map()
Map.centerObject(roi, 9)
Map.addLayer(dnbr_simple, {"min": 0, "max": 0.5, "palette": ["white", "yellow", "red", "black"]}, "Simple dNBR (Pre vs Post)")
Map.addLayer(
    dnbr_simple.updateMask(burn_mask),
    {"min": 0.2, "max": 0.5, "palette": ["yellow", "red", "black"]},
    "Burned Areas Mask (dNBR > 0.2)"
)
#Map.addLayer(roi, {}, "Pulang Pisau Boundary")
Map


Map(center=[-2.7158640396563944, 113.95461205914583], controls=(WidgetControl(options=['position', 'transparen…

In [ ]:
# Burn mask (Only show areas with dNBR > 0.2)
burn_mask = dnbr_simple.gt(0.2)

# Visualization
Map = geemap.Map()
Map.centerObject(roi, 9)
Map.addLayer(dnbr_simple, {"min": 0, "max": 0.5, "palette": ["white", "yellow", "red", "black"]}, "Simple dNBR (Pre vs Post)")
Map.addLayer(
    dnbr_simple.updateMask(burn_mask),
    {"min": 0.2, "max": 0.5, "palette": ["yellow", "red", "black"]},
    "Burned Areas Mask (dNBR > 0.2)"
)
#Map.addLayer(roi, {}, "Pulang Pisau Boundary")
Map